# Exploration and Exploitation

In [1]:
using PGFPlots
using Interact
using Reactive
using Distributions
include("helpers.jl")
include("bandits.jl");

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/c558bafc1ddf1f1075011ff5557dcbb87cc44e3c-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/c558bafc1ddf1f1075011ff5557dcbb87cc44e3c-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

## Multi-Armed Bandit Problems

In [2]:
using Random
Random.seed!(2)
arms = 3
b = Bandit(arms)
banditTrial(b)

Widget{:button,Int64}(OrderedDict{Symbol,Any}(), Observable{Int64}("ob_01", 0, Any[SyncCallback(Scope("knockout-component-1d80053b-dc52-4811-9132-7947a79f0195", Node{DOM}(DOM(:html, :button), Any["Arm 1"], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:style=>Dict{String,Any}()), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}(#= circular reference @-6 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/shushmanchoudhury/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"clicks\\\":0}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"clicks\"].subscribe((function (val){!(this.valueFromJulia[\"clicks\"]) ? (WebIO.setval({\"name\":\"clicks\",\"scope\":\"knockout-component-1d80053b-dc52-4811-9132-7947a79f0195\",\"id\":\"ob_01\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"clicks\"]=false}),self)]\n        \n    }\n    self.model = new AppViewModel();\n    self.valueFromJulia = {};\n    for (var key in json_data) {\n        self.valueFromJulia[key] = false;\n    }\n    ko.applyBindings(self.model, self.dom);\n}\n")]),"clicks"=>Any[JSString("(function (val){return (val!=this.model[\"clicks\"]()) ? (this.valueFromJulia[\"clicks\"]=true, this.model[\"clicks\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:1), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0))), SyncCallback(Scope("knockout-component-1d80053b-dc52-4811-9132-7947a79f0195", Node{DOM}(DOM(:html, :button), Any["Arm 1"], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:style=>Dict{String,Any}()), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}(#= circular reference @-7 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmancho

Observable{Any}("ob_03", "0 wins out of 0 tries (NaN percent)", Any[])

Widget{:button,Int64}(OrderedDict{Symbol,Any}(), Observable{Int64}("ob_05", 0, Any[SyncCallback(Scope("knockout-component-4343019d-7438-475e-9ba2-10e28b9c62ae", Node{DOM}(DOM(:html, :button), Any["Arm 2"], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:style=>Dict{String,Any}()), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}(#= circular reference @-6 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/shushmanchoudhury/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"clicks\\\":0}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"clicks\"].subscribe((function (val){!(this.valueFromJulia[\"clicks\"]) ? (WebIO.setval({\"name\":\"clicks\",\"scope\":\"knockout-component-4343019d-7438-475e-9ba2-10e28b9c62ae\",\"id\":\"ob_05\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"clicks\"]=false}),self)]\n        \n    }\n    self.model = new AppViewModel();\n    self.valueFromJulia = {};\n    for (var key in json_data) {\n        self.valueFromJulia[key] = false;\n    }\n    ko.applyBindings(self.model, self.dom);\n}\n")]),"clicks"=>Any[JSString("(function (val){return (val!=this.model[\"clicks\"]()) ? (this.valueFromJulia[\"clicks\"]=true, this.model[\"clicks\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:1), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0))), SyncCallback(Scope("knockout-component-4343019d-7438-475e-9ba2-10e28b9c62ae", Node{DOM}(DOM(:html, :button), Any["Arm 2"], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:style=>Dict{String,Any}()), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}(#= circular reference @-7 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmancho

Observable{Any}("ob_07", "0 wins out of 0 tries (NaN percent)", Any[])

Widget{:button,Int64}(OrderedDict{Symbol,Any}(), Observable{Int64}("ob_09", 0, Any[SyncCallback(Scope("knockout-component-6ed0f77a-cc8a-4bcf-9ef9-c812c269837b", Node{DOM}(DOM(:html, :button), Any["Arm 3"], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:style=>Dict{String,Any}()), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}(#= circular reference @-6 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/shushmanchoudhury/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"clicks\\\":0}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"clicks\"].subscribe((function (val){!(this.valueFromJulia[\"clicks\"]) ? (WebIO.setval({\"name\":\"clicks\",\"scope\":\"knockout-component-6ed0f77a-cc8a-4bcf-9ef9-c812c269837b\",\"id\":\"ob_09\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"clicks\"]=false}),self)]\n        \n    }\n    self.model = new AppViewModel();\n    self.valueFromJulia = {};\n    for (var key in json_data) {\n        self.valueFromJulia[key] = false;\n    }\n    ko.applyBindings(self.model, self.dom);\n}\n")]),"clicks"=>Any[JSString("(function (val){return (val!=this.model[\"clicks\"]()) ? (this.valueFromJulia[\"clicks\"]=true, this.model[\"clicks\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:1), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0))), SyncCallback(Scope("knockout-component-6ed0f77a-cc8a-4bcf-9ef9-c812c269837b", Node{DOM}(DOM(:html, :button), Any["Arm 3"], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:style=>Dict{String,Any}()), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}(#= circular reference @-7 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmancho

Observable{Any}("ob_11", "0 wins out of 0 tries (NaN percent)", Any[])

Widget{:togglebuttons,Any}(OrderedDict{Symbol,Any}(:options=>Observable{Array{String,1}}("ob_13", ["Hide", "Show"], Any[#g#15{##16#17{UnionAll,Observable{Any}},Tuple{Observable{Array{String,1}}}}(##16#17{UnionAll,Observable{Any}}(Vals2Idxs, Observable{Any}("ob_14", ["Hide", "Show"], Any[##119#122{Bool,ObservablePair{Any,Any}}(false, ObservablePair{Any,Any}(Observable{Any}("ob_15", "Hide", Any[##24#27{##117#120{Observable{Any},typeof(get)},Observable{Any},Array{Function,1}}(##117#120{Observable{Any},typeof(get)}(Observable{Any}(#= circular reference @-8 =#), get), Observable{Any}("ob_16", 1, Any[##25#28{##118#121{Observable{Any}},Observable{Any},Array{Function,1}}(##118#121{Observable{Any}}(Observable{Any}(#= circular reference @-11 =#)), Observable{Any}(#= circular reference @-6 =#), Function[##24#27{##117#120{Observable{Any},typeof(get)},Observable{Any},Array{Function,1}}(#= circular reference @-5 =#), ##25#28{##118#121{Observable{Any}},Observable{Any},Array{Function,1}}(#= circular reference @-2 =#)]), SyncCallback(Scope("knockout-component-d0f7fdb6-7326-43de-80e1-607112eb46cd", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["True Params"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :button), Any[Node{DOM}(DOM(:html, :label), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text : key")), 0)], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"click: function () {\$root.index(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'button is-fullwidth' : true}")), 1)], Dict{Symbol,Any}(:attributes=>("data-bind"=>"foreach : options_js"),:className=>"field is-grouped has-addons is-oneline is-centered"), 2)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 3), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[], Dict{Symbol,Any}(), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 9), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("options_js"=>(Observable{Any}("ob_17", OrderedDict[OrderedDict{String,Any}("key"=>"Hide","val"=>1,"id"=>"idH9cXgqkZ"), OrderedDict{String,Any}("key"=>"Show","val"=>2,"id"=>"idXDmbxAHa")], Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "options_js")))]), nothing),"index"=>(Observable{Any}(#= circular reference @-6 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/shushmanchoudhury/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringi

Observable{Any}("ob_18", "", Any[])

## Bayesian Model Estimation

In [3]:
Random.seed!(3)
arms = 2
b = Bandit(arms)
banditEstimation(b)

Widget{:button,Int64}(OrderedDict{Symbol,Any}(), Observable{Int64}("ob_22", 0, Any[SyncCallback(Scope("knockout-component-acfbe99d-0761-4af8-972d-4be099b9999f", Node{DOM}(DOM(:html, :button), Any["Arm 1"], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:style=>Dict{String,Any}()), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}(#= circular reference @-6 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/shushmanchoudhury/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"clicks\\\":0}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"clicks\"].subscribe((function (val){!(this.valueFromJulia[\"clicks\"]) ? (WebIO.setval({\"name\":\"clicks\",\"scope\":\"knockout-component-acfbe99d-0761-4af8-972d-4be099b9999f\",\"id\":\"ob_22\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"clicks\"]=false}),self)]\n        \n    }\n    self.model = new AppViewModel();\n    self.valueFromJulia = {};\n    for (var key in json_data) {\n        self.valueFromJulia[key] = false;\n    }\n    ko.applyBindings(self.model, self.dom);\n}\n")]),"clicks"=>Any[JSString("(function (val){return (val!=this.model[\"clicks\"]()) ? (this.valueFromJulia[\"clicks\"]=true, this.model[\"clicks\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:1), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0))), SyncCallback(Scope("knockout-component-acfbe99d-0761-4af8-972d-4be099b9999f", Node{DOM}(DOM(:html, :button), Any["Arm 1"], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:style=>Dict{String,Any}()), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}(#= circular reference @-7 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmancho

Observable{Any}("ob_24", "0 wins out of 0 tries (NaN percent)", Any[])

Observable{Any}("ob_26", Axis(PGFPlots.Plots.Plot[Linear(Real[0.0 0.010101 … 0.989899 1.0; 1.0 1.0 … 1.0 1.0], "none", nothing, nothing, "Beta(1, 1)", nothing, nothing)], nothing, nothing, nothing, nothing, nothing, nothing, nothing, 0, 1, 0, nothing, nothing, nothing, nothing, nothing, nothing, "15cm", "10cm", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, "axis"), Any[])

Widget{:button,Int64}(OrderedDict{Symbol,Any}(), Observable{Int64}("ob_28", 0, Any[SyncCallback(Scope("knockout-component-1854f8a4-6525-4484-a321-f4a30bce434d", Node{DOM}(DOM(:html, :button), Any["Arm 2"], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:style=>Dict{String,Any}()), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}(#= circular reference @-6 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/shushmanchoudhury/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"clicks\\\":0}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"clicks\"].subscribe((function (val){!(this.valueFromJulia[\"clicks\"]) ? (WebIO.setval({\"name\":\"clicks\",\"scope\":\"knockout-component-1854f8a4-6525-4484-a321-f4a30bce434d\",\"id\":\"ob_28\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"clicks\"]=false}),self)]\n        \n    }\n    self.model = new AppViewModel();\n    self.valueFromJulia = {};\n    for (var key in json_data) {\n        self.valueFromJulia[key] = false;\n    }\n    ko.applyBindings(self.model, self.dom);\n}\n")]),"clicks"=>Any[JSString("(function (val){return (val!=this.model[\"clicks\"]()) ? (this.valueFromJulia[\"clicks\"]=true, this.model[\"clicks\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:1), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0))), SyncCallback(Scope("knockout-component-1854f8a4-6525-4484-a321-f4a30bce434d", Node{DOM}(DOM(:html, :button), Any["Arm 2"], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:style=>Dict{String,Any}()), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}(#= circular reference @-7 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmancho

Observable{Any}("ob_30", "0 wins out of 0 tries (NaN percent)", Any[])

Observable{Any}("ob_32", Axis(PGFPlots.Plots.Plot[Linear(Real[0.0 0.010101 … 0.989899 1.0; 1.0 1.0 … 1.0 1.0], "none", nothing, nothing, "Beta(1, 1)", nothing, nothing)], nothing, nothing, nothing, nothing, nothing, nothing, nothing, 0, 1, 0, nothing, nothing, nothing, nothing, nothing, nothing, "15cm", "10cm", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, "axis"), Any[])

Widget{:togglebuttons,Any}(OrderedDict{Symbol,Any}(:options=>Observable{Array{String,1}}("ob_34", ["Hide", "Show"], Any[#g#15{##16#17{UnionAll,Observable{Any}},Tuple{Observable{Array{String,1}}}}(##16#17{UnionAll,Observable{Any}}(Vals2Idxs, Observable{Any}("ob_35", ["Hide", "Show"], Any[##119#122{Bool,ObservablePair{Any,Any}}(false, ObservablePair{Any,Any}(Observable{Any}("ob_36", "Hide", Any[##24#27{##117#120{Observable{Any},typeof(get)},Observable{Any},Array{Function,1}}(##117#120{Observable{Any},typeof(get)}(Observable{Any}(#= circular reference @-8 =#), get), Observable{Any}("ob_37", 1, Any[##25#28{##118#121{Observable{Any}},Observable{Any},Array{Function,1}}(##118#121{Observable{Any}}(Observable{Any}(#= circular reference @-11 =#)), Observable{Any}(#= circular reference @-6 =#), Function[##24#27{##117#120{Observable{Any},typeof(get)},Observable{Any},Array{Function,1}}(#= circular reference @-5 =#), ##25#28{##118#121{Observable{Any}},Observable{Any},Array{Function,1}}(#= circular reference @-2 =#)]), SyncCallback(Scope("knockout-component-627a043c-ad4a-4c47-b48d-f4b1327aa9e1", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["True Params"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :button), Any[Node{DOM}(DOM(:html, :label), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text : key")), 0)], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"click: function () {\$root.index(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'button is-fullwidth' : true}")), 1)], Dict{Symbol,Any}(:attributes=>("data-bind"=>"foreach : options_js"),:className=>"field is-grouped has-addons is-oneline is-centered"), 2)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 3), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[], Dict{Symbol,Any}(), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 9), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("options_js"=>(Observable{Any}("ob_38", OrderedDict[OrderedDict{String,Any}("key"=>"Hide","val"=>1,"id"=>"idKflpqFmw"), OrderedDict{String,Any}("key"=>"Show","val"=>2,"id"=>"idCZ1WNxqK")], Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "options_js")))]), nothing),"index"=>(Observable{Any}(#= circular reference @-6 =#), nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/shushmanchoudhury/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/shushmanchoudhury/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/shushmanchoudhury/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringi

Observable{Any}("ob_39", "", Any[])

## Ad Hoc Exploration Strategies

In [4]:
# Select random action with probability eps, otherwise greedy
struct EpsGreedy <: BanditPolicy
    eps::Real 
end
function arm(b::EpsGreedy, s::BanditStatistics)
    if rand() < b.eps
        D = DiscreteUniform(1, numArms(s))
        return rand(D)
    else
        return argmax(winProbabilities(s))
    end
end;

In [5]:
# Select arm with probability proportional to exp(precision*winprobability)
struct SoftMax <: BanditPolicy
    precision::Real 
end
function arm(b::SoftMax, s::BanditStatistics)
    p = exp.(b.precision * winProbabilities(s))
    p = p / sum(p)
    D = Categorical(p)
    return rand(D)
end;

In [6]:
# Select arm with highest alpha upper confidence bound
struct IntervalExploration <: BanditPolicy
    alpha::Real
end
function arm(b::IntervalExploration, s::BanditStatistics)
    i = argmax([quantile(Beta(s.numWins[i] + 1, s.numTries[i] - s.numWins[i] + 1), b.alpha) for i in 1:length(s.numWins)]) 
end;

In [7]:
steps = 50
iterations = 1000
bandit = Bandit(collect(0.1:0.2:1))
# bandit = Bandit(collect(1:-0.2:0.1))
epsgreedy = 0.1
softmax = 2
interval = 0.05
@manipulate for epsgreedy in 0:0.1:1, softmax in 0:2:40, interval in 0.5:0.05:1
    epsGreedyResults = simulateAverage(bandit, EpsGreedy(epsgreedy), steps=steps, iterations = iterations)./collect(1:steps)
    softMaxResults = simulateAverage(bandit, SoftMax(softmax), steps=steps, iterations = iterations)./collect(1:steps)
    intervalResults = simulateAverage(bandit, IntervalExploration(interval), steps=steps, iterations = iterations)./collect(1:steps)
    Axis([
        Plots.Linear(epsGreedyResults, legendentry="eps greedy", style="very thick", mark="none"),
        Plots.Linear(softMaxResults, legendentry="softmax", style="very thick", mark="none"),  
        Plots.Linear(intervalResults, legendentry="interval", style="very thick", mark="none")  
        ], style="legend pos=south east", ymin=0, ymax=1, xmin=0, xmax=steps, xlabel="Pulls", ylabel="Average success")
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:epsgreedy=>Widget{:slider,Float64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64}("ob_45", 0, Any[SyncCallback(Scope("knockout-component-4cbc7e3b-4d7e-4174-8034-33cb19be64c9", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["epsgreedy"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>11,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("formatted_vals"=>(Observable{Any}("ob_43", ["0.0", "0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9", "1.0"], Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "formatted_vals")))]), nothing),"changes"=>(Observable{Int64}(#= circular reference @-6 =#), nothing),"formatted_value"=>(Observable{String}("ob_44", "0.5", Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "formatted_value")))]), nothing),"index"=>(Observable{Any}("ob_42", 6, Any[##25#28{##70#73,Observable{Float64},Array{Function,1}}(##70#73(Box(0.0:0.1:1.0)), Observable{Float64}("ob_41", 0.5, Any[##24#27{##68#71,Observable{Any},Array{Function,1}}(##68#71(Box(0.0:0.1:1.0)), Observable{Any}(#= circular reference @-6 =#), Function[##24#27{##68#71,Observable{Any},Array{Function,1}}(#= circular reference @-2 =#), ##25#28{##70#73,Observable{Float64},Array{Function,1}}(#= circular reference @-5 =#)]), #g#15{##16#17{##30#32,Observable{Any}},Tuple{Observable{Float64},Observable{Int64},Observable{Float64}}}(##16#17{##30#32,Observable{Any}}(##30#32(), Observable{Any}("ob_53", Axis(PGFPlots.Plots.Plot[Linear(Real[1.0 2.0 … 49.0 50.0; 0.259 0.329 … 0.627816 0.6288], "none", nothing, "very thick", "eps greedy", nothing, nothing), Linear(Real[1.0 2.0 … 49.0 50.0; 0.478 0.5355 … 0.802367 0.80328], "none", nothing, "very thick", "softmax", nothing, nothing), Linear(Real[1.0 2.0 … 49.0 50.0; 0.083 0.1725 … 0.772898 0.77504], "none", nothing, "very thick", "interval", nothing, nothing)], nothing, "Pulls", nothing, "Average success", nothing, nothing, nothing, 0, 50, 0, 1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, "legend pos=south east", nothing, nothing, nothing, nothing, nothing, nothing, nothing, "axis"), Any[])), (Observable{Float64}(#= circular reference @-4 =#), Observable{Int64}("ob_46", 20, Any[SyncCallback(Scope("knockout-component-206eb13c-2300-41e4-8ba2-889221bbfa6a", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["softmax"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>40,:min=>0,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change 